# The Plan

0. Make sure you have all tools installed. mafft, trimal, iqtree at the very least.
1. Get a set of sequences to build a tree of.
2. Align these to each other.
4. Trim the alignment, removing gaps.
3. Build a fast phylogentic tree.
4. Build a thorough phylogenetic tree.


# Composing your fasta

Let's look at what we have

In [1]:
tree

.
├── analyses
│   ├── 1kP_LAR_orthogroup_random700_guidev3_fasttrees
│   │   └── aligned-mafft-linsi_trim-gt6-seq80
│   │       ├── 1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_fasttree.log
│   │       ├── 1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_fasttree.stderr
│   │       └── 1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_fasttree.tree
│   └── 1kP_LAR_orthogroup_random700_guidev3_trees
│       └── aligned-mafft-linsi_trim-gt6-seq80
│           ├── 1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000
│           ├── 1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000.bionj
│           ├── 1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000.ckp.gz
│           ├── 1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000.contree
│    

In [80]:
inseq=1kP_LAR_orthogroup

In [3]:
head data/$inseq.fasta

>KTWL-2000328-Kaliphora_madagascariensis
GGTGYIGKFLVEASAKAGHPTFAFAREITVSDPLKGKLVESFKNSGVTLVHGDLYDHSSLVKAIKQVDVVISTVGQGQLNDQGKIIDAIKEAGNVKKFYPSEFGVDVDRQHAVEPAKFTFTGKAQFRRAIEAAGIPHTFLVSNAFAGYVLPTLGQPGVSAPPHDKVVILGDGNAKAVVNEEHDIGTFTIKTVDDPRTLNKIVYIRPPKNTVSFNEIVAIWERTIGKTLEKEYVSEEQLLKQIKEAPPPLNYMLSIYHAYLVKGDSANFAIEPSGVEASELYPDVNYTTVEEYI
>QSNJ-2064790-Taiwania_cryptomerioides
KSKVLIFGATGYMGHFIAIASLDSDHPTFVFIRPSSLEDPARAQILSNLRSSGIHFITGSLEDHESLVRAIQGMDVVISVLGGGQIMEQLKIIDAIKQVGTVKRFLPSEFGHDVDRAEPVEPALGFYESKRRIRRATEEAKIGYTYICCNSIAGWPYHYHTHPSKMFPPTDKIHIYGNGNIKAYFMTGKDIGSYTMRAIEDPRTLNKKLHFRPADNFLTMNELAVMWQKQLGKKLPMAVISEEDLLKCARANVMPASIVAALTHDIFIKGCQYNFSIEGDEDEEACQLYPDIKYTKLEDYFSSF
>URCP-2006931-Athyrium_filix_femina
MASKSKVLVIGATGYIGQHIAQAGPAQGHPTFVLVRPSSLSSKATLIHSFKLAGITILEGSLDDYESLVAAIKQVDVVISALGSNTLDQLKIVEAIKEVGTIKRFFPSEFGNDVDRVKALEPASQVFGLKVQVRRAIEEAGIPYTYVVSNGFAGYFLRSLLQPGLQSPPRDKVSIYGSGDVKFITASEEDIGAYTIKAVDDPRTLNKTLHIRPAANIVTINSLVELWEKLIGKTLEKVTVTAEDIVKQIQETPFPNNIFPAILHDIVIQGDHCNFELGPKDVEAT

In [93]:
grep '>' data/$inseq.fasta -c

7449


In [7]:
n=700
grep '>' data/$inseq.fasta | tr -d '>' | shuf | head -n $n > data/"$inseq"_random"$n".txt

shuf: write error: Broken pipe
shuf: write error


In [94]:
grep '>'  ../LAR_tree/alignments_raw/LAR_orthogroup_LARselectionExtraFernsAndLycophytes_guide2_aligned-mafft-linsi.fasta | tr -d '>' | tail -n +49 > data/"$inseq"_random"$n".txt

In [4]:
wc -l ./data/"$inseq"_random700.txt

698 ./data/1kP_LAR_orthogroup_random700.txt


Let's add all Azolla stuff in there, but first see what is there already

In [11]:
grep -i azolla data/"$inseq"_random"$n".txt

LAR-like_Azolla_filiculoides
CVEG-2005710-Azolla_cf_caroliniana
CVEG-2016335-Azolla_cf_caroliniana
CVEG-2016336-Azolla_cf_caroliniana
CVEG-2005709-Azolla_cf_caroliniana
AZOLLAFILICULOIDES


In [17]:
grep -i azolla data/"$inseq".fasta

>CVEG-2005710-Azolla_cf._caroliniana
>CVEG-2016335-Azolla_cf._caroliniana
>CVEG-2016336-Azolla_cf._caroliniana
>CVEG-2005709-Azolla_cf._caroliniana
>AZOLLAFILICULOIDES


Remove double guide sequence AZOLLAFILICULOIDES

In [25]:
sed -i '/AZOLLAFILICULOIDES/d' data/"$inseq".fasta

In [26]:
head data/"$inseq"_random"$n".txt

Klefl_v1_0-kfl00402_0160-Klebsormidium_flaccidum
AKXB-2007298-Phaeomegaceros_coriaceus
AKXB-2067781-Phaeomegaceros_coriaceus
JKKI-2057061-Lobomonas_rostrata
Klefl_v1_0-kfl00034_0150-Klebsormidium_flaccidum
AKXB-2064674-Phaeomegaceros_coriaceus
RCBT-2175720-Sphagnum_palustre
RSOF-2008002-Glaucosphaera_vacuolata
AKXB-2001848-Phaeomegaceros_coriaceus
BAZF-2006540-Chaetopeltis_orbicularis


In [27]:
/opt/rnaseq/bin/extract_sequence_by_id.pl --ID=<(sort data/"$inseq"_random"$n".txt | uniq) \
                                          --fileformat=txt \
                                          --sequences=<(cat data/$inseq.fasta | tr -d '.')\
                                          --outfile=data/"$inseq"_random"$n".fasta

In [28]:
grep '>' -c data/"$inseq"_random"$n".fasta

666


In [20]:
n=700
grep -i azolla data/"$inseq"_random"$n".fasta

>CVEG-2005710-Azolla_cf_caroliniana
>CVEG-2016335-Azolla_cf_caroliniana
>CVEG-2016336-Azolla_cf_caroliniana
>CVEG-2005709-Azolla_cf_caroliniana
>AZOLLAFILICULOIDES


In [29]:
ls data

1kP_LAR_orthogroup.fasta                    alignments_raw
1kP_LAR_orthogroup_random700.fasta          alignments_trimmed
1kP_LAR_orthogroup_random700_guidev3.fasta  Erbils_guide_v3.fasta
1kP_LAR_orthogroup_random700.txt


In [30]:
head data/1kP_LAR_orthogroup_random700.fasta

>YSRZ-2016241-Fouqueria_macdougalli
MQKSKVLIVGGTGYLGRRLVKASLALEHETYVLRRPEIGVDIDKAQMLLSFKKEGAHLVPGSFDDHRSLVDAVKMVDVVIRSHQILLQLKLVDAIKEAGNVKRFLPSEFGTDPAKMENAMEPGRVTFDDKMVVRKAIEDAGIPHTYVSANCFAGYFIGGLCQPGQIFPSKDHVVLLGDGNQKAIYVDEDDIATYTIKTIDDPQTINKTLYLRPPENILSQREVIQIWEKLIGKELQKSSISSEQFLATMKEQDYAEQVGLTHYYHVCYEGCLANFEIGDE
>HJMP-2011848-Astragalus_membranaceus
TLNKTLLLRPSANTLTFNELVSLWENKIKSTIHKIYIPEEKLLKTIQETPFPGNLISALCHSILVKGDSANFDIDPSFCVEGTGLYPEVKYTTVDNYLNAFV
>OSIP-2125743-Garcinia_livingstonei
SPNILEACLGRTLIIGSTGFIGGFVAEASVETGRPTYLLIRPGTSKISTIRSLQEKGATILYGSINDQDIIERVLREHKIEYVISAVGGVSTVDQVNLVKAIKSVGSVKRFLPSEFGHDIDRTEPVEPGLTMYREKRMVRRAVEEAGIPYTYICCNSIAAWPYHDNIHPADVLPPLDRFLIYGDGSVKAYFVAGTDIGRFTMKVLDDDRTLNKTVHFQPPSNLLNVNELASLWEDKIGRKLPRFTVTEADLLTAAKEMPIPKSIVAALTHDIFINGCQINYTMDKEIDVEICSLYPDTPLRTIDQCFDDFLAKVVANRKPIKNPAAKN
>TVCU-2006094-Ochna_mossambicensis
PNVSEAGRTLVVGSTGFIGRFVTDACIDSGIPTYVLVRSGPVSASKDSAIKSFQDKGATIIHGSMDDQELMMKILKDYKIEFVISAVGGDSILDQIKLVNAVKEVGTVKRFLPSEFGHDIDRADPVEPGLTMYNEKRKVRRC

In [100]:
head data/Erbils_guide_v3.fasta 

>Ocimum tenuiflorum EGS (AOC97444)
MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSTKTALLNEFQSMGAIIVKGEMGEHEKLV
RLMKKVDVVISAVAIPQVLEQLKIVEAIKVAGNIKRFLPSDFGVEEDRISALPPFEACIEKKRIIRRAIE
EANIPYTYVSANCFASYFINYLLHPSDPKDEITVYGTGEVKFAMNYEQDIGLYTIKVATDPRTLNRVVIY
RPSTNITTQLELISKWEKKIGRNFKKIHIPEEEIVALTKELPEPENIPVAILHCVFIKGVTMSYDFKEND
VEASTLYPELKFTTIDELLDIFLHDPPPPASAAF

>Clarkia breweri IGS (ABR24112)
MEKIIIYGGTGYIGKFMVRASLSFSHPTFIYARPLTPDSTPSSVQLREEFRSMGVTIIEGEMEEHEKMVS
VLRQVDVVISALSVPMYPSQLLIIDAIKAAGNIKRFLPSEFGSEEDRIKPLPPFESVLEKKRIIRRAIEA


In [32]:
grep '>' data/Erbils_verified_sequences_v2_nonredundantwithv3.fasta

>EGS Ocimum gratissimum (Koeduka et al. 2018)
>EGS Ocimum kilimandscharicum (Koeduka et al. 2018)
>EGS Ocimum basillicum (Koeduka et al. 2018)
>EGS Gymnadenia conopsea (Koeduka et al. 2018)
>EGS Gymnadenia odoratissima (Koeduka et al. 2018)
>PLR Linum perenne (Koeduka et al. 2018)
>P(L)R Arabidopsis thaliana (Koeduka et al. 2018)
>LAR Vitis vinifera (Koeduka et al. 2018)
>LAR Desmodium uncinatum
>LAR Malus domestica (Koeduka et al. 2018)
>LAR Fragaria x ananassa (Koeduka et al. 2018)
>IFR-LIKE Oryza sativa (Kim et al. 2009)
>IFR-LIKE Ginkgo biloba (Hua et al. 2013)
>LAR Pinus taeda
>LAR Picea abies
>LAR Azolla caroliniana (CVEG-2016335)


In [31]:
grep '>' data/Erbils_guide_v3.fasta

>Ocimum tenuiflorum EGS (AOC97444)
>Clarkia breweri IGS (ABR24112)
>Clarkia breweri EGS1 (ABR24113)
>Piper regnellii PPS (AHA90809)
>Larrea tridentata PPS (AHA90806)
>Pimpinella anisum AIS (ACL13526)
>Petunia hybrida IGS (ABD17322)
>Gymnadenia densiflora EGS (AKB11749)
>Fragaria ananassa EGS1 (AGV02007)
>Larrea tridentata APS (AHA90804)
>Lotus japonicus PTR (BAF34844)
>Clarkia breweri EGS2 (ABR24114)
>Populus trichorpa PCBER (CAA06707)
>Forsythia intermedia PCBER (AAF64174)
>Petunia hybrida EGS (ABR24115)
>Nicotiana tabacum PCBER (BAG84267)
>Fragaria ananassa EGS2 (AGV02008)
>Piper regnellii APS (AHA90807)
>Pinus taeda PCBER (AAC32591)
>Tsuga heterophylla PCBER (AAF64179)
>Cicer arietinum IFR (Q00016)
>Pisum sativum IFR (P52576)
>Medicago sativa IFR (CAA41106)
>Tsuga heterophylla PLR (AAF64184)
>Thuja plicata PLR (AAF63509)
>Forsythia intermedia PLR (AAC49608)
>Linum album PLR (CAH60857)
>Desmodium uncinatum LAR (Q84V83)
>Fragaria ananassa LAR (ABH07785)
>Selaginella moellendorffii PLR

In [36]:
cat data/Erbils_guide_v3.fasta > data/1kP_LAR_orthogroup_random700_guidev4.fasta

In [38]:
cat data/Erbils_verified_sequences_v2_nonredundantwithv3.fasta >> data/1kP_LAR_orthogroup_random700_guidev4.fasta

In [39]:
echo "\n" >> data/1kP_LAR_orthogroup_random700_guidev4.fasta

In [40]:
cat data/1kP_LAR_orthogroup_random700.fasta >> data/1kP_LAR_orthogroup_random700_guidev4.fasta

In [41]:
grep '>' -c ./data/1kP_LAR_orthogroup_random700_guidev4.fasta

718


In [42]:
inseq="$inseq"_random"$n"_guidev4
echo $inseq

1kP_LAR_orthogroup_random700_guidev4


In [43]:
head data/$inseq.fasta

>Ocimum tenuiflorum EGS (AOC97444)
MEENGMKSKILIFGGTGYIGNHMVKGSLKLGHPTYVFTRPNSTKTALLNEFQSMGAIIVKGEMGEHEKLV
RLMKKVDVVISAVAIPQVLEQLKIVEAIKVAGNIKRFLPSDFGVEEDRISALPPFEACIEKKRIIRRAIE
EANIPYTYVSANCFASYFINYLLHPSDPKDEITVYGTGEVKFAMNYEQDIGLYTIKVATDPRTLNRVVIY
RPSTNITTQLELISKWEKKIGRNFKKIHIPEEEIVALTKELPEPENIPVAILHCVFIKGVTMSYDFKEND
VEASTLYPELKFTTIDELLDIFLHDPPPPASAAF

>Clarkia breweri IGS (ABR24112)
MEKIIIYGGTGYIGKFMVRASLSFSHPTFIYARPLTPDSTPSSVQLREEFRSMGVTIIEGEMEEHEKMVS
VLRQVDVVISALSVPMYPSQLLIIDAIKAAGNIKRFLPSEFGSEEDRIKPLPPFESVLEKKRIIRRAIEA


# Aligning

### Either run everything one by one...

First with MAFFT

In [27]:
mafft --help

------------------------------------------------------------------------------
  MAFFT v7.429 (2019/Jul/1)
  https://mafft.cbrc.jp/alignment/software/
  MBE 30:772-780 (2013), NAR 30:3059-3066 (2002)
------------------------------------------------------------------------------
High speed:
  % mafft in > out
  % mafft --retree 1 in > out (fast)

High accuracy (for <~200 sequences x <~2,000 aa/nt):
  % mafft --maxiterate 1000 --localpair  in > out (% linsi in > out is also ok)
  % mafft --maxiterate 1000 --genafpair  in > out (% einsi in > out)
  % mafft --maxiterate 1000 --globalpair in > out (% ginsi in > out)

If unsure which option to use:
  % mafft --auto in > out

--op # :         Gap opening penalty, default: 1.53
--ep # :         Offset (works like gap extension penalty), default: 0.0
--maxiterate # : Maximum number of iterative refinement, default: 0
--clustalout :   Output: clustal format, default: fasta
--reorder :      Outorder: aligned, default: input order
--quiet :       

: 1

This is probably the most acurate mafft setting, which is turned off by default in normal or auto mafft for alignments bigger than 200 sequences. Since We're moving to a publication quality tree, let's do is this way and see. 

In [123]:
conda activate phylogenetics

(phylogenetics) 

: 1

In [124]:
#rm "./data/alignments_raw/$inseq"_aligned-mafft-linsi.fasta
if    [ ! -d ./data/alignments_raw/ ]
then  mkdir  ./data/alignments_raw
fi
if    [ ! -f "./data/alignments_raw/$inseq"_aligned-mafft.fasta ]
then  linsi --thread 6 data/$inseq.fasta > "./data/alignments_raw/$inseq"_aligned-mafft-linsi.fasta
fi

(phylogenetics) (phylogenetics) outputhat23=16
treein = 0
compacttree = 0
stacksize: 8192 kb
rescale = 1
All-to-all alignment.
tbfast-pair (aa) Version 7.455
alg=L, model=BLOSUM62, 2.00, -0.10, +0.10, noshift, amax=0.0
6 thread(s)

outputhat23=16
Loading 'hat3.seed' ... 
done.
Writing hat3 for iterative refinement
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00
tbutree = 1, compacttree = 0
Constructing a UPGMA tree ... 
  130 / 137
done.

Progressive alignment ... 
STEP   135 /136 (thread    3) 
Reallocating (by thread 2) ..done. *alloclen = 1927
STEP   136 /136 (thread    2) 
done.
tbfast (aa) Version 7.455
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
6 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 6
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 16
sueff_global = 0.100000
nadd = 16
Loading 'hat3' ... done.
rescale = 1

  130 / 137
Segment   1/  1    1- 628
004-0270-1 (thread    1) better        
Converged2.
done
dvtditr (aa) Versio

: 1

In [125]:
ls ./data/alignments_raw

1kP_LAR_orthogroup_manual-selection-1_guidev4_aligned-mafft-linsi.fasta
1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi.fasta
1kP_LAR_orthogroup_random700_guidev4_aligned-mafft-linsi.fasta
(phylogenetics) 

: 1

In [126]:
head ./data/alignments_raw/"$inseq"_aligned-mafft-linsi.fasta

>Ocimum tenuiflorum EGS (AOC97444)
M-----------------------------------------------------------
-----------------------EENGMKSKILIFGGTGYIGNHMVKGSLK------LGH
-----------------------------PTYVFTRP----------------NSTKTAL
LNEFQSMGAIIVKGEMG-E----------------HEKLVRLMK----------------
---------KVDVVISAVAIPQ----------------VLEQLKIVEAIKVA--------
---G--NIKRFLPSDF-------GVEEDR--IS-ALP-PFEA----CIEKKRIIRRAIEE
ANIPYTYVSANCFASYFINYLLHPS---------DPK----D-EITVYGTGEVKFAMNYE
QDIGLYTIKVATDPRTLNRVVIYRPSTNIT---TQLELISKWEKK------IGRNFKKIH
IPEEEIVALT--KELPEP-E-NIPVAILHCVF-IKGVTMSYDFKEN-D-VEASTLYPELK
(phylogenetics) 

: 1

## Alignment trimming

odds are, your alignment is quite gappy which may confuse tree building algorithms. Often it is better to remove gappy columns in your alignment. Let's have a look at this with `trimAl`. Short for 'trim alignment' No Artificial intelegence stuff going on here.

In [50]:
trimal -h


trimAl v1.4.rev15 build[2013-12-17]. 2009-2013. Salvador Capella-Gutierrez and Toni Gabaldón.

trimAl webpage: http://trimal.cgenomics.org

This program is free software: you can redistribute it and/or modify 
it under the terms of the GNU General Public License as published by 
the Free Software Foundation, the last available version.

Please cite:
		trimAl: a tool for automated alignment trimming in large-scale phylogenetic analyses.
		Salvador Capella-Gutierrez; Jose M. Silla-Martinez; Toni Gabaldon.
		Bioinformatics 2009, 25:1972-1973.

Basic usage
	trimal -in <inputfile> -out <outputfile> -(other options).

Common options (for a complete list please see the User Guide or visit http://trimal.cgenomics.org):

    -h                       Print this information and show some examples.
    --version                Print the trimAl version.

    -in <inputfile>          Input file in several formats (clustal, fasta, NBRF/PIR, nexus, phylip3.2, phylip).

    -compareset <inputfile>  In


9) Selection of sequences to be deleted from the alignment. Start in 0

   trimal -in <inputfile> -out <outputfile> -selectseqs { 2,4,8-12 } 

10) Select the 5 most representative sequences from the alignment

   trimal -in <inputfile> -out <outputfile> -clusters 5 

(phylogenetics) 

: 1

I'm trying some manual trimming as alternative

In [127]:
mkdir data/alignments_trimmed 2> /dev/null

for a in "data/alignments_raw/$inseq"_aligned*.fasta
do  appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
    if    [ ! -f data/alignments_trimmed/"$inseq"_"$appendix"_trim-gt6-seq80.fasta ]
    then  echo "trimming alignment $a"
          sed -i 's/ /_/g' $a
          trimal -in $a   \
                 -out data/alignments_trimmed/"$inseq"_"$appendix"_trim-gt6-seq80.fasta \
                 -gt .6 \
                 -seqoverlap 80 \
                 -resoverlap .6 \
                 -htmlout data/alignments_trimmed/"$inseq"_"$appendix"_trim-gt6-seq80.html 
    fi
done

(phylogenetics) (phylogenetics) trimming alignment data/alignments_raw/1kP_LAR_orthogroup_manual-selection-1_guidev4_aligned-mafft-linsi.fasta


(phylogenetics) 

: 1

In [52]:
ls data/alignments_trimmed

1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6.html
1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80.fasta
1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80.html
1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq90.html
1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt7.html
1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt8.html
1kP_LAR_orthogroup_random700_guidev4_aligned-mafft-linsi_trim-gt6-seq80.fasta
1kP_LAR_orthogroup_random700_guidev4_aligned-mafft-linsi_trim-gt6-seq80.html
(phylogenetics) 

: 1

In [53]:
head data/alignments_trimmed/"$inseq"_aligned*.fasta

>Ocimum_tenuiflorum_EGS_(AOC97444)
KILIFGGTGYIGNHMVKGSLKLGHPTYVFTRP----TKTALLNEFQSMGAIIVKGEMGEH
EKLVRLMKKVDVVISAVAIPQVLEQLKIVEAIKVAGNIKRFLPSDFGVEEDRISALPPFE
ACIEKKRIIRRAIEEANIPYTYVSANCFASYFINYLLHPSD---PKDEITVYGTGEVKFA
MNYEQDIGLYTIKVATDPRTLNRVVIYRPSTNITTQLELISKWEKKIGRNFKKIHIPEEE
IVALTKELPEPENIPVAILHCVFIKGVTMSYDFKENDVEASTLYPELKFTTIDELLDIF
>Clarkia_breweri_IGS_(ABR24112)
KIIIYGGTGYIGKFMVRASLSFSHPTFIYARPLTPSSSVQLREEFRSMGVTIIEGEMEEH
EKMVSVLRQVDVVISALSVPMYPSQLLIIDAIKAAGNIKRFLPSEFGSEEDRIKPLPPFE
SVLEKKRIIRRAIEAAELPYTYVSANCFGAYFVNYLLHPSN---RDDDIVIYGTGETKFV
(phylogenetics) 

: 1

In [129]:
conda deactivate

# Tree building
We'll make fast trees (not so acurate, no bootstraps, but fast) (optional) and we'll make "propper trees" using the amazing `iqtree`

## fasttree

In [ ]:
rm -rf ./data/LAR_orthogroup_fasttrees

In [ ]:
ls analyses/1kP_LAR_orthogroup_random700_guidev3_fasttrees

In [ ]:
for a in data/alignments_trimmed/"$inseq"_aligned*.fasta
do echo "making a fasttree of file $a"
   appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
   echo $appendix
   if   [ ! -d analyses/"$inseq"_fasttrees ]
   then mkdir  analyses/"$inseq"_fasttrees
   fi
   if   [ ! -d analyses/"$inseq"_fasttrees/"$appendix" ]
   then mkdir  analyses/"$inseq"_fasttrees/"$appendix" 
        fasttree -log analyses/"$inseq"_fasttrees/"$appendix"/"$inseq"_"$appendix"_fasttree.log \
                $a \
                > analyses/"$inseq"_fasttrees/"$appendix"/"$inseq"_"$appendix"_fasttree.tree \
                2> analyses/"$inseq"_fasttrees/"$appendix"/"$inseq"_"$appendix"_fasttree.stderr &
    fi
done

In [ ]:
tail analyses/"$inseq"_fasttrees/"$appendix"/"$inseq"_"$appendix"_fasttree.log

In [ ]:
ls analyses/"$inseq"_fasttrees/"$appendix"/

## IQtree 

Now let's loop over all made aligments, making a tree. Choose your parameters wisely, this can take a long time

 * Are doing a first testrun? Have a look at the -fast option
 * No time to wait for propper bootstraps, use the ultrafast bootstrap option, -bb 1000 (Don't believe nodes with less than 95% bootstrap
 * Writing for your amazing manuscript? Do propper bootstraps! -b 1000
 
Model finder, this is the best feature of iqtree. 
Evolution can happen in a lot of ways, and iqtree takes this into account. 
Use `-m TEST` to use modelfinder, or `-m MFP` for extended modelfinding (for your publication quality tree). 
Modelfinder, especially the extended version, can take a long time to calculate. 
So if you have done it once for a specific alignment, don't do it twice. ;)

Now have a look at these examples and the manual:

### iqtree ultra fast bootstrap

In [56]:
ls data/alignments_trimmed/"$inseq"_aligned*gt*.fasta

data/alignments_trimmed/1kP_LAR_orthogroup_random700_guidev4_aligned-mafft-linsi_trim-gt6-seq80.fasta


In [60]:
grep 'Best-fit model'  ./analyses/1kP_LAR_orthogroup_random700_guidev3_trees/aligned-mafft-linsi_trim-gt6-seq80/1kP_LAR_orthogroup_random700_guidev3_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000

Best-fit model: LG+R9 chosen according to BIC


In [135]:
for a in data/alignments_trimmed/"$inseq"_aligned*gt*.fasta
do echo "making a tree of file $a"
   appendix=$(echo $a | cut -d '/' -f 3- | sed "s/$inseq\_//" | sed "s/.fasta//")
   echo $appendix
   if   [ ! -d analyses/"$inseq"_trees ]
   then mkdir  analyses/"$inseq"_trees
   fi
   if   [ ! -d analyses/"$inseq"_trees/"$appendix" ]
   then mkdir  analyses/"$inseq"_trees/"$appendix" 
   fi
   if   [ ! -f analyses/"$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-bb1000.tree ]
   then     nice iqtree -s $a \
            -b 1000 \
            -nt AUTO \
            -ntmax 12  \
            -pre analyses/"$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b1000 \
            2>  analyses/"$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b1000 \
            >  analyses/"$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b1000 \
            -m MFP && \
            cat analyses/"$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b1000.log | mail -s LARguide4 laura.w.dijkhuizen@gmail.com &
   fi
done

making a tree of file data/alignments_trimmed/1kP_LAR_orthogroup_manual-selection-1_guidev4_aligned-mafft-linsi_trim-gt6-seq80.fasta
aligned-mafft-linsi_trim-gt6-seq80
[1] 13959


In [130]:
ls analyses/"$inseq"_trees/aligned*/

1kP_LAR_orthogroup_manual-selection-1_guidev4_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000
1kP_LAR_orthogroup_manual-selection-1_guidev4_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000.log
1kP_LAR_orthogroup_manual-selection-1_guidev4_aligned-mafft-linsi_trim-gt6-seq80_iqtree-bb2000-shalrt2000.model.gz


In [140]:
tail -n 40 analyses/"$inseq"_trees/aligned-mafft-*_trim-gt*/"$inseq"_*_iqtree-b10*log

Log-likelihood of BIONJ tree: -26054.931
--------------------------------------------------------------------
|             INITIALIZING CANDIDATE TREE SET                      |
--------------------------------------------------------------------
Generating 98 parsimony trees... 3.243 second
Computing log-likelihood of 98 initial trees ... 3.834 seconds
Current best score: -26018.011

Do NNI search on 20 best initial trees
Estimate model parameters (epsilon = 0.100)
BETTER TREE FOUND at iteration 1: -25937.370
Estimate model parameters (epsilon = 0.100)
BETTER TREE FOUND at iteration 2: -25929.496
Estimate model parameters (epsilon = 0.100)
BETTER TREE FOUND at iteration 7: -25914.412
Iteration 10 / LogL: -25979.020 / Time: 0h:0m:11s
Iteration 20 / LogL: -25955.453 / Time: 0h:0m:14s
Finish initializing candidate tree set (20)
Current best tree score: -25914.412 / CPU time: 13.561
Number of iterations: 20
--------------------------------------------------------------------
|           

In [151]:
grep START analyses/"$inseq"_trees/aligned-mafft-*_trim-gt*/"$inseq"_*_iqtree-b10*log -B 2 | grep 'Total' | cut -d ':' -f 2 | cut -d ' ' -f 2 

207.710
68.437
160.606
159.044
59.495
129.266
104.288
89.461
75.402
156.090
127.875
73.773
164.789
94.132
89.039
100.568
91.968
98.221
222.949
70.330
102.509
67.632
204.925
114.973
162.648
72.103
128.652
106.543
70.537
96.770
73.556
89.199
106.254
103.882
170.749
101.082
130.859
170.035
91.127
130.286
160.742
155.138
154.546
132.545
75.157
169.577
102.092
94.485
120.175
99.854
91.182
113.628
74.572
111.872
79.561
128.137
165.133
75.980
94.105
104.848
168.801
137.418
117.718
116.673
100.831
110.485
73.302
195.893
94.550
168.349
93.181
147.269
78.141
125.551
101.201
137.044
255.232
70.162
157.198
86.679
67.720
156.570
127.188
107.918
97.744
87.710
114.682
135.268
101.482
133.839
121.002
142.569
86.912
93.676
92.130
60.067
70.612
73.490
102.705
167.626
104.834
76.348
94.770
76.464
106.696
84.185
92.886
160.983
108.668
74.362
156.502
71.418
129.707
87.569
107.085


# tree storage

find the trees in iTOL: https://itol.embl.de/shared/lauradijkhuizen

# Refinement

Based on the analysis above, I'd like to make a tree of a subset of this tree. I'll be using: 
 * Erbils verified sequences
 * Representatives of the major clades in the LAR and PCBER section of the tree.
 * For those representatives I'll select several genera, and then take all sequences of that genus for my subset.
 
 These genera are
  * Eucalyptus
  * Pinus
  * Linum
  * Blechnum
  * Davallia 
  * Azolla
  * Dipteris
  * Ceratopteris
  * Osmunda
  * Quercus
  * Populus
  * Freycinetia 
  * Lendenbergia
  * Salix
  * Galax
  
  Nevermind, that's way too much. I'm selecting on these spefific key words
  
  * Azolla
  * Blechnum
  * Ceratopteris
  * Linum_tenuifolium
  * Eucalyptus_leucoxylon
  * Pinus_radiata
  * Salix_acutifolia
  

In [102]:
grep '>' data/$inseq.fasta | tr -d '>' | grep -f <(echo -e "Eucalyptus\nPinus\nLinum\nBlechnum\nDavallia\nAzolla\nDipteris\nCeratopteris\nOsmunda\nQuercus\nPopulus\nFreycinetia\nLendenbergia\nSalix\nGalax" ) | cut -f 3 -d '-'| sort | uniq -c

      4 Azolla_cf._caroliniana
     16 Blechnum_spicant
      9 BVOF
     10 Ceratopteris_thalictroides
     11 Davallia_fejeensis
     18 Dipteris_conjugata
     56 Eucalyptus_grandis
     12 Eucalyptus_leucoxylon
      9 Freycinetia_multiflora
     17 Galax_urceolata
      8 Linum_bienne
      5 Linum_flavum
      8 Linum_grandiflorum
      7 Linum_hirsutum
      6 Linum_leonii
     10 Linum_lewisii
      9 Linum_macraei
      8 Linum_strictum
      8 Linum_tenuifolium
     13 Osmunda_javanica
     12 Osmunda_sp.
     17 Osmundastrum_cinnamomeum
     21 Pinus_jeffreyi
     13 Pinus_parviflora
     22 Pinus_ponderosa
     22 Pinus_radiata
     62 Pinus_taeda
     22 Populus_trichocarpa
     11 POZS
     13 Quercus_shumardii
     13 Salix_acutifolia
      9 Salix_dasyclados
     10 Salix_eriocephala
      9 Salix_fargesii
      7 Salix_purpurea
      7 Salix_sachalinensis
     10 Salix_viminalis


In [106]:
grep '>' data/$inseq.fasta | tr -d '>' | grep -f <(echo -e "Azolla\nBlechnum\nCeratopteris\nLinum_tenuifolium\nEucalyptus_leucoxylon\nPinus_radiata\nSalix_acutifolia" ) | sort | uniq | wc -l

85


So that's 85 sequences from 7 genera + all ERbils Guide sequences, this will be fun. First extract this into a new txt and then a new fasta

In [108]:
grep '>' data/$inseq.fasta | tr -d '>' | grep -f <(echo -e "Azolla\nBlechnum\nCeratopteris\nLinum_tenuifolium\nEucalyptus_leucoxylon\nPinus_radiata\nSalix_acutifolia" ) | sort | uniq > data/1kP_LAR_orthogroup_manual-selection-1.txt

In [112]:
/opt/rnaseq/bin/extract_sequence_by_id.pl --ID=data/1kP_LAR_orthogroup_manual-selection-1.txt \
                                          --fileformat=txt \
                                          --sequences=<(cat data/$inseq.fasta )\
                                          --outfile=data/"$inseq"_manual-selection-1.fasta

In [113]:
grep '>'  data/"$inseq"_manual-selection-1.fasta -c

85


In [117]:
cat data/Erbils_guide_v3.fasta > data/"$inseq"_manual-selection-1_guidev4.fasta

In [118]:
echo "\n" >> data/"$inseq"_manual-selection-1_guidev4.fasta

In [119]:
cat data/Erbils_verified_sequences_v2_nonredundantwithv3.fasta >> data/"$inseq"_manual-selection-1_guidev4.fasta

In [120]:
cat data/"$inseq"_manual-selection-1.fasta >> data/"$inseq"_manual-selection-1_guidev4.fasta

In [121]:
grep '>' -c data/"$inseq"_manual-selection-1_guidev4.fasta

137


In [122]:
inseq="$inseq"_manual-selection-1_guidev4
echo $inseq

1kP_LAR_orthogroup_manual-selection-1_guidev4


Now I'm redoing the whole thing again from up-top.

stats after linsi mapping and trimal trimming

Selected Sequences:   100 /Selected Residues:     304
Deleted Sequences:     37 /Deleted Residues:      324

I'm quite happy with that, so let's start making a tree.

# other stuff


Manually set parameters for the script

Copy pasting and adapting the command above to do 
 * no modelfitting, I did that already
 * propper boostraps, just 200. (Minimum 100)
 * 12 threads
 * **new names**

In [ ]:
iqtree -s ./alignments_trimmed/"$inseq"_"$appendix".fasta \
       -b 200 \
       -nt AUTO \
       -ntmax 12 \
       -pre "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b200 \
       -m LG+R9 \
       >      "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b200.stdout \
       && cat "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b200.stdout \
       | mail -s 'LAR tree lauraZolla Desktop' laura &
       

In [ ]:
cat -n 80  "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-b200.stdout

### likelihood bootstrap thingy

http://www.iqtree.org/doc/Tutorial

https://www.ncbi.nlm.nih.gov/pubmed/16785212



In [ ]:
nice iqtree -s ./alignments_trimmed/"$inseq"_"$appendix".fasta \
       -bb   1000 \
       -alrt 1000 \
       -nt AUTO \
       -ntmax $(nproc) \
       -pre "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-bb1000-alrt1000 \
       -m LG+R9 \
       >      "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-bb1000-alrt1000.stdout \
       && cat "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-bb1000-alrt1000.stdout \
       | mail -s 'LAR tree alrt1000 lauraZolla Desktop' laura &
       

In [ ]:
tail -n 40 "$inseq"_trees/"$appendix"/"$inseq"_"$appendix"_iqtree-bb1000-alrt1000.stdout